In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from WindPy import w
from sklearn.linear_model import LinearRegression
import datetime
%matplotlib inline

In [27]:
w.start()

.ErrorCode=0
.Data=[Already conntected!]

In [28]:
end_date = datetime.datetime.today()
start_date = end_date - datetime.timedelta(200)

# 获取股票指数历史收益率

In [29]:
# codes = 'CI005917.WI,CI005919.WI,CI005018.WI,CI005916.WI' # 金融地产、消费、医药、科技
codes = '000934.SH,000931.CSI,000933.SH,000935.SH' # 中证金融地产、可选消费、必选医药、信息技术

In [30]:
data = w.wsd(codes, 'close', start_date, end_date)

In [31]:
df = pd.DataFrame(np.array(data.Data).T, index=data.Times, columns=data.Codes)

In [32]:
rdf = df.pct_change().dropna()

# 获取股票基金历史收益率

In [33]:
fund_df = pd.read_excel(u'../data/股票基金池.xlsx')
# fund_df = pd.read_excel(u'../data/嘉实股票基金池.xlsx')

In [34]:
fund_df

,代码,名称
0,159915.OF,易方达创业板ETF
1,001938.OF,中欧时代先锋A
2,450009.OF,国富中小盘
3,000595.OF,嘉实泰和
4,110015.OF,易方达行业领先
5,519736.OF,交银新成长
6,161005.OF,富国天惠精选成长A
7,159919.OF,嘉实沪深300ETF
8,169101.OF,东方红睿丰
9,002624.OF,广发优企精选


In [35]:
codes = fund_df[u'代码'].tolist()

In [36]:
codes

[u'159915.OF',
 u'001938.OF',
 u'450009.OF',
 u'000595.OF',
 u'110015.OF',
 u'519736.OF',
 u'161005.OF',
 u'159919.OF',
 u'169101.OF',
 u'002624.OF',
 u'160916.OF',
 u'519069.OF',
 u'000577.OF',
 u'163407.OF',
 u'100038.OF',
 u'001974.OF',
 u'000311.OF',
 u'000176.OF',
 u'000082.OF',
 u'070019.OF',
 u'159933.OF',
 u'000251.OF',
 u'005662.OF',
 u'040008.OF',
 u'159928.OF',
 u'110022.OF',
 u'162605.OF',
 u'006604.OF',
 u'000083.OF',
 u'110011.OF',
 u'001044.OF',
 u'070099.OF',
 u'512010.OF',
 u'110023.OF',
 u'001717.OF',
 u'003095.OF',
 u'000220.OF',
 u'005303.OF',
 u'000711.OF',
 u'159939.OF',
 u'001513.OF',
 u'001071.OF',
 u'003053.OF',
 u'519772.OF',
 u'519712.OF',
 u'001410.OF',
 u'004450.OF',
 u'002168.OF']

In [37]:
data = w.wsd(codes, 'nav', start_date, end_date)

In [38]:
fdf = pd.DataFrame(np.array(data.Data).T, index=data.Times, columns=data.Codes)

In [39]:
fdf = fdf.dropna(axis=1)

In [40]:
rfdf = fdf.pct_change().dropna()

In [41]:
codes = rfdf.columns.tolist()

In [42]:
print rfdf.shape
print rdf.shape

(133, 48)
(133, 4)


# 非负线性优化求解

In [43]:
# decomp = pd.DataFrame(index=codes, columns=[u'金融地产', u'消费', u'医药', u'科技'])
decomp = pd.DataFrame(index=codes, columns=[u'金融地产', u'可选消费', u'必选医药', u'科技'])
decomp.index.name = u'基金代码'

In [44]:
def objective(coef, args):
    n = coef.size
    X, y = args[0], args[1]
    target = np.dot(X, coef)
#     error = np.sum(np.square(target - y))
    error = np.sum(np.square(target - y)) + np.linalg.norm(coef, 1) # lasso error
    return error / n

def decomposition(code):
    TOLERANCE = 1e-10
    n = rdf.shape[1]
    init_weights = np.array([1. / n] * n)
    X = rdf.values
    y = rfdf[code].values
    
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0},
                   {'type': 'ineq', 'fun': lambda x: x})
    
    opt_res = minimize(fun=objective,
                       x0=init_weights,
                       args=[X, y],
                       constraints=constraints,
                       tol=TOLERANCE,
                       options={'disp': True})
    
    return opt_res.x

In [45]:
decomposition(codes[0])

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.250658387099
            Iterations: 16
            Function evaluations: 96
            Gradient evaluations: 16


array([0.10038226, 0.0906707 , 0.30396225, 0.50498479])

In [46]:
for c in codes:
    weight = decomposition(c)
    decomp.loc[c] = weight

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.250658387099
            Iterations: 16
            Function evaluations: 96
            Gradient evaluations: 16
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.251029669345
            Iterations: 19
            Function evaluations: 114
            Gradient evaluations: 19
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.252588794781
            Iterations: 15
            Function evaluations: 90
            Gradient evaluations: 15
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.25671706892
            Iterations: 15
            Function evaluations: 90
            Gradient evaluations: 15
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.250918024043
            Iterations: 16
            Function evaluations: 96
  

In [47]:
decomp[u'基金名称'] = fund_df[fund_df[u'代码'].isin(codes)][u'名称'].tolist()

In [48]:
if u'市值占净值' in fund_df.columns:
    decomp[u'市值占净值'] = fund_df[fund_df[u'代码'].isin(codes)][u'市值占净值'].tolist()

In [49]:
decomp = decomp[decomp.columns[4:].tolist() + decomp.columns[:4].tolist()]

In [50]:
decomp

,基金名称,金融地产,可选消费,必选医药,科技
基金代码,,,,,
159915.OF,易方达创业板ETF,0.100382,0.0906707,0.303962,0.504985
001938.OF,中欧时代先锋A,0.26655,8.25349e-18,0.350086,0.383364
450009.OF,国富中小盘,0.561637,0.142899,0.295464,-7.65718e-18
000595.OF,嘉实泰和,-3.59142e-18,-1.10836e-17,0.723771,0.276229
110015.OF,易方达行业领先,0.297016,0.17664,0.435488,0.0908559
519736.OF,交银新成长,0.567893,0.0226586,0.409448,-1.57548e-19
161005.OF,富国天惠精选成长A,0.285559,0.162332,0.408373,0.143736
159919.OF,嘉实沪深300ETF,0.529027,0.215252,0.185887,0.0698339
169101.OF,东方红睿丰,0.275452,0.430497,0.0959008,0.198151


In [51]:
if u'市值占净值' in fund_df.columns:
    decomp = decomp.append(pd.DataFrame([['', '',
                           np.dot(decomp[u'金融地产'], fund_df[u'市值占净值']) / fund_df[u'市值占净值'].sum(),
                           np.dot(decomp[u'可选消费'], fund_df[u'市值占净值']) / fund_df[u'市值占净值'].sum(),
                           np.dot(decomp[u'必选医药'], fund_df[u'市值占净值']) / fund_df[u'市值占净值'].sum(),
                           np.dot(decomp[u'科技'], fund_df[u'市值占净值']) / fund_df[u'市值占净值'].sum(), 
                           ]], index=[u'组合'], columns=decomp.columns))

In [52]:
decomp.to_excel(u'../data/股票基金收益率分解.xlsx')
# decomp.to_excel(u'../data/嘉实股票基金收益率分解.xlsx')